In [121]:
!pip install lxml

In [122]:
import bs4
import pandas as pd
from urllib.request import urlopen
import lxml

def HR_game_schedule(year):
    url = "https://www.hockey-reference.com/leagues/NHL_" + str(year) + "_games.html"
    html = urlopen(url).read()
    soup = bs4.BeautifulSoup(html, "html5lib")
    
    tb = soup.find("table", { "id":"games"})
    column_headers = ['Date', 'Away', 'AwayG', 'Home', 'HomeG', 'OTSO', 'Attendance', 'LOG', 'Notes']
    
    records = []
    for idy, row in enumerate(tb.findAll('tr')):
        if idy == 0:
            continue
        cols = row.findAll(['td','th'])
        record = {}
        for idx, col in enumerate(cols):
            record[column_headers[idx]] = col.text.strip()
        records.append(record)
    
    df = pd.DataFrame.from_dict(records)
    
    df.Away = df.Away.apply(str.upper)
    df.Away = [''.join(i.split()) for i in df.Away]
    df.Home = df.Home.apply(str.upper)
    df.Home = [''.join(i.split()) for i in df.Home]
    return df

In [123]:
print(HR_game_schedule(2019))

     Attendance                 Away AwayG        Date                 Home  \
0        17,562         ANAHEIMDUCKS     5  2018-10-03        SANJOSESHARKS   
1        19,589    MONTREALCANADIENS     2  2018-10-03    TORONTOMAPLELEAFS   
2        18,870        CALGARYFLAMES     2  2018-10-03     VANCOUVERCANUCKS   
3        18,506         BOSTONBRUINS     0  2018-10-03   WASHINGTONCAPITALS   
4        19,070         BOSTONBRUINS     4  2018-10-04        BUFFALOSABRES   
5        18,680     NEWYORKISLANDERS     2  2018-10-04   CAROLINAHURRICANES   
6        18,086        MINNESOTAWILD     1  2018-10-04    COLORADOAVALANCHE   
7        18,532       ARIZONACOYOTES     0  2018-10-04          DALLASSTARS   
8        19,515  COLUMBUSBLUEJACKETS     3  2018-10-04      DETROITREDWINGS   
9        17,117   NASHVILLEPREDATORS     3  2018-10-04       NEWYORKRANGERS   
10       15,858    CHICAGOBLACKHAWKS     4  2018-10-04       OTTAWASENATORS   
11       18,627   WASHINGTONCAPITALS     6  2018-10-

In [134]:
import pandas as pd
import numpy as np

In [135]:
pd.options.display.max_rows = 500

In [136]:
class Team:

    def team_mean(self, data, column):
        res = np.asarray(data[column].values.tolist())
        res = res.astype(np.int64)
        return res, np.mean(res)

    def count_win(self, data, place, what_res):
        res = data[what_res].value_counts()
        print("cw", what_res)
        win = res[place]
        try:
            sum = res[0] + res[1] + res[2]
            tied = res['tied']
        except:
            tied = 0
            sum = res[0] + res[1]
        lose = sum - win - tied
        print(win, tied, lose)
        return [win, tied, lose]

    def count_win_result(self, what_res):
        print("cwr", what_res)
        return add1(self.count_win(self.match_guest, 'Guest', what_res),
                    self.count_win(self.match_home, 'Home', what_res))

    def __init__(self, name, df):
        self.name = name
        self.df = df
        self.match_home = pd.concat([self.df[self.df.Team_Home == self.name]], ignore_index=True)
        self.match_guest = pd.concat([self.df[self.df.Team_Guest == self.name]], ignore_index=True)
        self.match_all = pd.concat([self.match_guest, self.match_home], ignore_index=True)

        self.stats_fst = self.count_win_result('Res_Fst')
        self.stats_snd = self.count_win_result('Res_Snd')
        self.stats_trd = self.count_win_result('Res_Trd')
        self.stats_res_with_OT = self.count_win_result('Result_with_OT')
        self.stats_res_no_OT = self.count_win_result('Result_NO_OT')


In [137]:
data_match = pd.DataFrame(columns=['Team_Home', 'Team_Guest', 'Result_NO_OT', 'Result_with_OT', 'Goal_Home',
                                   'Goal_Guest', 'Fst_Home', 'Fst_Guest', 'Res_Fst', 'Snd_Home', 'Snd_Guest',
                                   'Res_Snd', 'Trd_Home', 'Trd_Guest', 'Res_Trd'])

teams = ['Айлендерс', 'Анахайм', 'Аризона', 'Баффало', 'Бостон', 'Ванкувер', 'Вашингтон', 'Вегас', 'Виннипег',
         'Даллас', 'Детройт', 'Калгари', 'Каролина', 'Коламбус', 'Колорадо', 'Лос-Анджелес', 'Миннесота', 'Монреаль',
         'Нью-Джерси', 'Нэшвилл', 'Оттава', 'Питтсбург', 'Рейнджерс', 'Сан-Хосе', 'Сент-Луис', 'Тампа-Бэй',
         'Торонто', 'Филадельфия', 'Флорида', 'Чикаго', 'Эдмонтон']

def rename(team_russian):
    if team_russian == 'Анахайм':
        return 'Anaheim Ducks'

    if team_russian == 'Сан-Хосе':
        return 'San Jose Sharks'

    if team_russian == 'Виннипег':
        return 'Winnipeg Jets'

    if team_russian == 'Вашингтон':
        return 'Washington Capitals'

    if team_russian == 'Ванкувер':
        return 'Vancouver Canucks'

    if team_russian == 'Вегас':
        return 'Vegas Golden Knights'

    if team_russian == 'Торонто':
        return 'Toronto Maple Leafs'

    if team_russian == 'Тампа-Бэй':
        return 'Tampa Bay Lightning'

    if team_russian == 'Сент-Луис':
        return 'St Louis Blues'

    if team_russian == 'Питтсбург':
        return 'Pittsburgh Penguins'

    if team_russian == 'Филадельфия':
        return 'Philadelphia Flyers'

    if team_russian == 'Оттава':
        return 'Ottawa Senators'

    if team_russian == 'Рейнджерс':
        return 'New York Rangers'

    if team_russian == 'Айлендерс':
        return 'New York Islanders'

    if team_russian == 'Нью-Джерси':
        return 'New Jersey Devils'

    if team_russian == 'Нэшвилл':
        return 'Nashville Predators'

    if team_russian == 'Монреаль':
        return 'Montreal Canadiens'

    if team_russian == 'Миннесота':
        return 'Minnesota Wild'

    if team_russian == 'Лос-Анджелес':
        return 'Los Angeles Kings'

    if team_russian == 'Флорида':
        return 'Florida Panthers'

    if team_russian == 'Эдмонтон':
        return 'Edmonton Oilers'

    if team_russian == 'Детройт':
        return 'Detroit Red Wings'

    if team_russian == 'Даллас':
        return 'Dallas Stars'

    if team_russian == 'Коламбус':
        return 'Columbus Blue Jackets'

    if team_russian == 'Колорадо':
        return 'Colorado Avalanche'

    if team_russian == 'Чикаго':
        return 'Chicago Blackhawks'

    if team_russian == 'Каролина':
        return 'Carolina Hurricanes'

    if team_russian == 'Баффало':
        return 'Buffalo Sabres'

    if team_russian == 'Калгари':
        return 'Calgary Flames'

    if team_russian == 'Бостон':
        return 'Boston Bruins'

    if team_russian == 'Аризона':
        return 'Arizona Coyotes'


In [138]:
def add1(seq1, seq2):
    for i in range(len(seq1)):
        seq1[i] = seq1[i] + seq2[i]
    return seq1

In [139]:
import sys
def result_game(home, guest, is_overtime=False):
    if is_overtime:
        return 'Tied'
    if home > guest:
        return 'Home'
    if home == guest:
        return 'Tied'
    return 'Guest'


def check_team(str, teams):
    for team in teams:
        if team in str:
            return True
    return False


def no_OT(str):
    All = str[0].split(':')
    Fst = str[1].split(':')
    Snd = str[2].split(':')
    Trd = str[3].split(':')
    return (All[0].replace(':', ''), All[1].replace(':', ''), Fst[0].replace(':', ''), Fst[1].replace(':', ''),
            Snd[0].replace(':', ''), Snd[1].replace(':', ''), Trd[0].replace(':', ''), Trd[1].replace(':', ''))


def OT(str):
    All = str[0].split(':')
    Fst = str[2].split(':')
    Snd = str[3].split(':')
    Trd = str[4].split(':')
    return (All[0].replace(':', ''), All[1].replace(':', ''), Fst[0].replace(':', ''), Fst[1].replace(':', ''),
            Snd[0].replace(':', ''), Snd[1].replace(':', ''), Trd[0].replace(':', ''), Trd[1].replace(':', ''))


def data_append(data, home, guest, goal_home, goal_guest, fst_home,
                fst_guest, snd_guest, snd_home, trd_home, trd_guest, is_overtime):
    data = data.append({'Team_Home': rename(home), 'Team_Guest': rename(guest), 'Goal_Home': goal_home, 
                            'Goal_Guest': goal_guest,
                            'Fst_Home': fst_home, 'Fst_Guest': fst_guest, 'Snd_Home': snd_home, 'Snd_Guest': snd_guest,
                            'Trd_Home': trd_home, 'Trd_Guest': trd_guest,
                            'Result_NO_OT': result_game(goal_home, goal_guest, is_overtime),
                            'Result_with_OT': result_game(goal_home, goal_guest),
                            'Res_Fst': result_game(fst_home, fst_guest),
                            'Res_Snd': result_game(snd_home, snd_guest),
                            'Res_Trd': result_game(trd_home, trd_guest)}, ignore_index=True)
    return data


def win_name_time(data, home_goal, guest_goal):
    name_time = home_goal.split('_')[0] + '_win_name'
    col = pd.DataFrame(columns=[name_time])
    for index, row in data.iterrows():
        if row[home_goal] > row[guest_goal]:
            col = col.append({name_time: row['Team_Home']}, ignore_index=True)
        if row[home_goal] == row[guest_goal]:
            col = col.append({name_time: 'Tied'}, ignore_index=True)
        if row[home_goal] < row[guest_goal]:
            col = col.append({name_time: row['Team_Guest']}, ignore_index=True)
    result = pd.concat([data, col], axis=1)
    return result


def win_name_all(df):
    df = win_name_time(df, 'Fst_Home', 'Fst_Guest')
    df = win_name_time(df, 'Snd_Home', 'Snd_Guest')
    df = win_name_time(df, 'Trd_Home', 'Trd_Guest')
    return df


def match_interpretate(str, team, data):
    if check_team(str, team):
        str_old = str
        str = str.replace(',', '').replace('(', '').replace(')', '')
        t = str.split('–')
        home = t[0].replace(' ', '')
        guest = t[1].replace(' ', '')
        score = t[2][1:].split(' ')
        if 'ОТ' not in str_old and 'бул' not in str_old:
            score = no_OT(score)
            
            goal_home, goal_guest, fst_home, fst_guest = score[0], score[1], score[2], score[3]
            snd_home, snd_guest, trd_home, trd_guest = score[4], score[5], score[6], score[7]
            is_overtime = False

        if 'ОТ' in str_old or 'бул' in str_old:
            score = OT(score)
            goal_home, goal_guest, fst_home, fst_guest = score[0], score[1], score[2], score[3]
            snd_home, snd_guest, trd_home, trd_guest = score[4], score[5], score[6], score[7]
            is_overtime = True
        data = data_append(data, home, guest, goal_home, goal_guest, fst_home,
                            fst_guest, snd_guest, snd_home, trd_home, trd_guest, is_overtime)
        return data
    else:
        return data


#print(data_match[20:50])
#wash = Team('Washington Capitals', data_match)
# print(wash.match_home.columns)
#print(wash.match_all)
#print(wash.stats_res_with_OT)

In [140]:
f = open('True_data.txt', encoding='utf8')
#true_data = open('True_data.txt', 'w', encoding='utf8')
for line in f:
    data_match = match_interpretate(line, teams, data_match)
f.close()
data_match = win_name_all(data_match)

In [141]:
data_match

,Team_Home,Team_Guest,Result_NO_OT,Result_with_OT,Goal_Home,Goal_Guest,Fst_Home,Fst_Guest,Res_Fst,Snd_Home,Snd_Guest,Res_Snd,Trd_Home,Trd_Guest,Res_Trd,Fst_win_name,Snd_win_name,Trd_win_name
0,Washington Capitals,Boston Bruins,Home,Home,7,0,2,0,Home,4,0,Home,1,0,Home,Washington Capitals,Washington Capitals,Washington Capitals
1,Toronto Maple Leafs,Montreal Canadiens,Tied,Home,3,2,1,1,Tied,1,1,Tied,0,0,Tied,Tied,Tied,Tied
2,Vancouver Canucks,Calgary Flames,Home,Home,5,2,1,0,Home,0,0,Tied,4,2,Home,Vancouver Canucks,Tied,Vancouver Canucks
3,San Jose Sharks,Anaheim Ducks,Guest,Guest,2,5,1,1,Tied,1,1,Tied,0,3,Guest,Tied,Tied,Anaheim Ducks
4,Pittsburgh Penguins,Washington Capitals,Tied,Home,7,6,2,3,Guest,3,1,Home,1,2,Guest,Washington Capitals,Pittsburgh Penguins,Washington Capitals
5,Carolina Hurricanes,New York Islanders,Tied,Guest,1,2,0,0,Tied,0,1,Guest,1,0,Home,Tied,New York Islanders,Carolina Hurricanes
6,New York Rangers,Nashville Predators,Guest,Guest,2,3,0,0,Tied,1,1,Tied,1,2,Guest,Tied,Tied,Nashville Predators
7,St Louis Blues,Winnipeg Jets,Guest,Guest,1,5,0,1,Guest,0,0,Tied,1,4,Guest,Winnipeg Jets,Tied,Winnipeg Jets
8,Colorado Avalanche,Minnesota Wild,Home,Home,4,1,1,1,Tied,1,0,Home,2,0,Home,Tied,Colorado Avalanche,Colorado Avalanche
9,Vegas Golden Knights,Philadelphia Flyers,Guest,Guest,2,5,1,2,Guest,0,3,Guest,1,0,Home,Philadelphia Flyers,Philadelphia Flyers,Vegas Golden Knights


In [142]:
data_match

,Team_Home,Team_Guest,Result_NO_OT,Result_with_OT,Goal_Home,Goal_Guest,Fst_Home,Fst_Guest,Res_Fst,Snd_Home,Snd_Guest,Res_Snd,Trd_Home,Trd_Guest,Res_Trd,Fst_win_name,Snd_win_name,Trd_win_name
0,Washington Capitals,Boston Bruins,Home,Home,7,0,2,0,Home,4,0,Home,1,0,Home,Washington Capitals,Washington Capitals,Washington Capitals
1,Toronto Maple Leafs,Montreal Canadiens,Tied,Home,3,2,1,1,Tied,1,1,Tied,0,0,Tied,Tied,Tied,Tied
2,Vancouver Canucks,Calgary Flames,Home,Home,5,2,1,0,Home,0,0,Tied,4,2,Home,Vancouver Canucks,Tied,Vancouver Canucks
3,San Jose Sharks,Anaheim Ducks,Guest,Guest,2,5,1,1,Tied,1,1,Tied,0,3,Guest,Tied,Tied,Anaheim Ducks
4,Pittsburgh Penguins,Washington Capitals,Tied,Home,7,6,2,3,Guest,3,1,Home,1,2,Guest,Washington Capitals,Pittsburgh Penguins,Washington Capitals
5,Carolina Hurricanes,New York Islanders,Tied,Guest,1,2,0,0,Tied,0,1,Guest,1,0,Home,Tied,New York Islanders,Carolina Hurricanes
6,New York Rangers,Nashville Predators,Guest,Guest,2,3,0,0,Tied,1,1,Tied,1,2,Guest,Tied,Tied,Nashville Predators
7,St Louis Blues,Winnipeg Jets,Guest,Guest,1,5,0,1,Guest,0,0,Tied,1,4,Guest,Winnipeg Jets,Tied,Winnipeg Jets
8,Colorado Avalanche,Minnesota Wild,Home,Home,4,1,1,1,Tied,1,0,Home,2,0,Home,Tied,Colorado Avalanche,Colorado Avalanche
9,Vegas Golden Knights,Philadelphia Flyers,Guest,Guest,2,5,1,2,Guest,0,3,Guest,1,0,Home,Philadelphia Flyers,Philadelphia Flyers,Vegas Golden Knights


In [143]:
teams_english = []
for team in teams:
    teams_english.append(rename(team))


print(teams_english)

table_result = pd.DataFrame(columns=['name', 'game', 'points', 'procent_points'])
for team in teams_english:
    what_team = Team(team, data_match)
    table_result = table_result.append({'name': team, 'game': what_team.how_many_game, 'points': what_team.points,
                                        'procent_points': what_team.procent_points}, ignore_index=True)
print(table_result)

['New York Islanders', 'Anaheim Ducks', 'Arizona Coyotes', 'Buffalo Sabres', 'Boston Bruins', 'Vancouver Canucks', 'Washington Capitals', 'Vegas Golden Knights', 'Winnipeg Jets', 'Dallas Stars', 'Detroit Red Wings', 'Calgary Flames', 'Carolina Hurricanes', 'Columbus Blue Jackets', 'Colorado Avalanche', 'Los Angeles Kings', 'Minnesota Wild', 'Montreal Canadiens', 'New Jersey Devils', 'Nashville Predators', 'Ottawa Senators', 'Pittsburgh Penguins', 'New York Rangers', 'San Jose Sharks', 'St Louis Blues', 'Tampa Bay Lightning', 'Toronto Maple Leafs', 'Philadelphia Flyers', 'Florida Panthers', 'Chicago Blackhawks', 'Edmonton Oilers']
cwr Res_Fst
cw Res_Fst
4 0 5
cw Res_Fst
6 0 3
cwr Res_Snd
cw Res_Snd
4 0 4
cw Res_Snd
5 0 4
cwr Res_Trd
cw Res_Trd
2 0 8
cw Res_Trd
3 0 5
cwr Result_with_OT
cw Result_with_OT
6 0 6
cw Result_with_OT
6 0 5
cwr Result_NO_OT
cw Result_NO_OT
4 0 6
cw Result_NO_OT
5 0 3


AttributeError: 'Team' object has no attribute 'how_many_game'